<a href="https://colab.research.google.com/github/Dilraj0/comp215/blob/main/labs/lab06_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 6
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * create a Watts-Strogatz graph both from scratch and from the Networkx module
  * measure the average clustering coefficient and path length of a network
  * visualize summative data of a graph


During this lab, you will be introduced to the following:
  * numpy arrays
  * local file i/o in Google Colab

(this lab is based on workbooks provided in Allen Downey's 'Think Complexity')

## Social Networking

This lab uses graphs to explore social networks using Facebook data.  In this lab you will create a simulated model of the network using a Watts-Strogatz graph and compare some properties of the simulated network and the real Facebook network.

In [ ]:
# put your imports here

In [46]:
import numpy as np
import networkx as nx
import random
from collections import deque

## Get the Facebook Data

Stanford Network Analysis Platform (SNAP) is a general purpose network analysis and graph mining library.  In previous labs, we have used APIs to access data.  For this lab, we will copy the data to a local file.  Download the ```facebook_combined.txt.gz``` file from [SNAP](https://snap.stanford.edu/data/egonets-Facebook.html), unzip it, and copy ```facebook_combined.txt``` to the ```Files``` folder in Colab.  

Look at the content of the file and read the SNAP webpage to understand what the data represents before moving on to the rest of the lab.

## Exercise 1: Make Facebook Graph

Write a function that reads the file, one edge per line, specified by the two integer node IDs given in each line of the file and returns a ```networkx``` graph representing the data.  You can do this with Python's built-in file handling, or you could use ```numpy```'s ```loadtxt``` function.  Write a unit test to check that the network has 4039 nodes and 88234 edges (as given in the Dataset Statistics on the SNAP site) and draw the Facebook network (this takes about a minute).


In [4]:
def read_facebook_graph_with_numpy(file_path):
    G = nx.Graph()  # creating empty graph

    data = np.loadtxt(file_path, dtype=int)
    edges = (tuple(edge) for edge in data)  # loading files and converting into tuples

    G.add_edges_from(edges)
    return G

# giving variable to file to use furthur
file_path = "/content/facebook_combined.txt"

# Creating the graph
facebook_graph = read_facebook_graph_with_numpy(file_path)

assert facebook_graph.number_of_nodes() == 4039
assert facebook_graph.number_of_edges() == 88234

print(f"Facebook Graph Loaded!")
print(f"Nodes: {facebook_graph.number_of_nodes()}, Edges: {facebook_graph.number_of_edges()}")

# Drawing the network of facebook
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(facebook_graph, seed=42)  # Layout for visualization
nx.draw(facebook_graph, pos, node_size=10, edge_color="black")
plt.title("Facebook Network Graph", fontsize=14)
plt.show()


Facebook Graph Loaded!
Nodes: 4039, Edges: 88234


KeyboardInterrupt: 

<Figure size 1000x800 with 0 Axes>

## Exercise 2: Clustering Coefficients

With larger graphs, it can take a long time to compute clustering coefficients and path lengths. We can estimate them by sampling without much loss of accuracy if the sample size is large enough.  Write a function that calculates the average clustering coeffient for a random subset of a N nodes in a network.  You may use the ```node_clustering``` and ```all_pairs``` functions from Chapter 5 of the textbook.  You may also use the ```numpy``` module to calculate the mean; note that there is a ```nanmean``` function.

Check that your clustering coeffients function gives a similar answer to the ```networkx``` ```average_clustering``` function.


In [52]:
# creating a function
def sample_clustering_coefficient(G, N=100):

    sample_nodes = random.sample(list(G.nodes()), min(N, len(G)))  # Sample N nodes
    clustering_values = [nx.clustering(G, node) for node in sample_nodes]  # Compute clustering coefficient for each
    return np.nanmean(clustering_values)  # Average clustering coefficient

# Loading the facebook data
file_path = "/content/facebook_combined.txt"
def load_facebook_graph(file_path):
    return nx.read_edgelist(file_path, nodetype=int)

# we can also use nx inbuild function so we can compare it after
def compare_clustering(G, N=100):
    sample_avg = sample_clustering_coefficient(G, N)
    networkx_avg = nx.average_clustering(G)

    print(f"Sampled Clustering Coefficient (N={N}): {sample_avg:.4f}")
    print(f"NetworkX Average Clustering Coefficient: {networkx_avg:.4f}")

file_path = "/content/facebook_combined.txt"
G = load_facebook_graph(file_path)
compare_clustering(G, N=1000)

Sampled Clustering Coefficient (N=1000): 0.6052
NetworkX Average Clustering Coefficient: 0.6055
Difference: 0.0003


## Exercise 3: Average Shortest Path Length

Write a function that calculates the average shortest path length for all pairs of nodes in a network.  You may use the ```shortest_path_dijkstra``` function from Chapter 5 of the textbook.  Using that function, it took my algorithm about 2 minutes to find the average shortest path over all pairs of nodes.


Check that your average shortest path length function gives a similar answer to the ```networkx``` ```average_shortest_path_length``` function.


In [53]:
def exact_average_shortest_path(G):
    return nx.average_shortest_path_length(G)

#creating function to estimate shortest path
def sample_path_lengths(G, nodes=None, trials=100):
    """Choose random pairs of nodes and compute the path length between them.
    G: Graph
    N: number of pairs to choose
    returns: list of path lengths
    """
    if nodes is None:
        nodes = list(G)
    else:
        nodes = list(nodes)

    pairs = np.random.choice(nodes, (trials, 2))  # Random pairs
    lengths = [nx.shortest_path_length(G, *pair) for pair in pairs]
    return lengths

def estimate_path_length(G, nodes=None, trials=1000):
    return np.mean(sample_path_lengths(G, nodes, trials))

# Loading Facebook Graph
def load_facebook_graph(file_path):
    return nx.read_edgelist(file_path, nodetype=int)

def compare_path_length(G, trials=1000):
    estimated_avg = estimate_path_length(G, trials=trials)
    exact_avg = nx.average_shortest_path_length(G)

    print(f"Estimated Average Shortest Path Length (Trials={trials}): {estimated_avg:.4f}")
    print(f"Exact NetworkX Average Shortest Path Length: {exact_avg:.4f}")

file_path = "/content/facebook_combined.txt"  #file path
G = load_facebook_graph(file_path)
compare_path_length(G, trials=1000)


Estimated Average Shortest Path Length (Trials=1000): 3.7410
Exact NetworkX Average Shortest Path Length: 3.6925


Here is a function from the textbook that takes a sample of path lengths to estimate the average shortest path length.  You may use this in the rest of the lab so that you don't need to wait for the whole full averaging algorithms above to run.

In [ ]:

def sample_path_lengths(G, nodes=None, trials=100):
    """Choose random pairs of nodes and compute the path length between them.
    G: Graph
    N: number of pairs to choose
    returns: list of path lengths
    """
    if nodes is None:
        nodes = list(G)
    else:
        nodes = list(nodes)

    pairs = np.random.choice(nodes, (trials, 2))
    lengths = [nx.shortest_path_length(G, *pair)
               for pair in pairs]
    return lengths

def estimate_path_length(G, nodes=None, trials=1000):
    return np.mean(sample_path_lengths(G, nodes, trials))

In the exercises above, you should have found that the Facebook network has an average clustering coefficient around 0.6 and an average shortest path length of around 3.7. Note that this corresponds to a 'degree of separation' of less than 6.   

## Exercise 4: WS Graph

Construct a WS graph with the same number of nodes and average degree as the Facebook network using the ```make_ws_graph``` function from Chapter 5.  Find the value of p (probability of rewire) that reproduces a clustering coefficient and average shortest path length of the Facebook network.  (Note that there is a ```nx.watts_strogatz_graph``` that you may use after you have demonstrated that you can create a WS graph using the functions from Chapter 5.).

What could this value of p tell you about the actual social network that this Facebook data represents?  (Think about what p means in the model and what that would represent in the data.)

In [ ]:
#accessing the facebook graph
edges = np.loadtxt("f/content/facebook_combined.txt", dtype=int)
G_facebook = nx.Graph()
G_facebook.add_edges_from(edges)

# cluster and avg shortest length
clustering_facebook = nx.average_clustering(G_facebook)
avg_shortest_path_facebook = nx.average_shortest_path_length(G_facebook)

n = len(G_facebook.nodes())
k = int(np.mean([deg for _, deg in G_facebook.degree()]))

best_p = None
min_diff = float('inf')

for p in np.linspace(0, 1, 100):
    G_ws = nx.watts_strogatz_graph(n, k, p)  # Create the WS graph with this p
    clustering_ws = nx.average_clustering(G_ws)
    avg_shortest_path_ws = nx.average_shortest_path_length(G_ws)

    # Calculate the difference between Facebook and WS graph properties
    diff = abs(clustering_ws - clustering_facebook) + abs(avg_shortest_path_ws - avg_shortest_path_facebook)

    # Track the best p (closest match)
    if diff < min_diff:
        min_diff = diff
        best_p = p

# getting output
print(f"The best p value is: {best_p}")

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(p_values, clustering_ws_values)
plt.xlabel("Rewiring Probability (p)")
plt.ylabel("Clustering Coefficient")

plt.subplot(1, 2, 2)
plt.plot(p_values, avg_shortest_path_ws_values)
plt.xlabel("Rewiring Probability (p)")
plt.ylabel("Average Shortest Path Length")

plt.tight_layout()
plt.show()
